# id가 4번인 사용자의 액션영화 평점 예측하기

In [2]:
# -*- coding: utf-8 -*-

%matplotlib inline

import time
import operator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Data Source : https://grouplens.org/datasets/movielens

rating_file_path = "../data/ml-1m/ratings.dat"
movie_file_path = "../data/ml-1m/movies.dat"
user_file_path = "../data/ml-1m/users.dat"

rating_data = pd.io.parsers.read_csv(rating_file_path, 
                                     names=['user_id', 'movie_id', 'rating', 'time'], delimiter='::')
movie_data = pd.io.parsers.read_csv(movie_file_path, 
                                    names=['movie_id', 'title', 'genre'], delimiter='::')
user_data = pd.io.parsers.read_csv(user_file_path, 
                                   names=['user_id', 'gender', 'age', 'occupation', 'zipcode'], delimiter='::')

In [4]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rating_data[['user_id', 'movie_id', 'rating']], reader)
train_data = data.build_full_trainset()

svd = SVD()
svd.fit(train_data)

# ID가 4인 사용자가 보지 않은 액션 영화를 찾습니다.
user_id = 4
user_unwatched_action_movies = movie_data[(~movie_data['movie_id'].isin(rating_data[rating_data['user_id'] == user_id]['movie_id'])) & movie_data['genre'].str.contains("Action")]

# 사용자가 보지 않은 액션 영화에 대한 평점을 예측합니다.
ratings = []
for _, movie in user_unwatched_action_movies.iterrows():
    estimated_rating = svd.predict(user_id, movie['movie_id']).est
    ratings.append(estimated_rating)

# 예측한 평점의 평균을 계산합니다.
estimated_mean_rating = sum(ratings) / len(ratings)
print("ID가 4인 사용자가 보지 않은 액션 영화에 대한 예측 평균 평점: {:.2f}".format(estimated_mean_rating))


ID가 4인 사용자가 보지 않은 액션 영화에 대한 예측 평균 평점: 3.32
